# Punto 5
Fake News identification in COVID public discussion. Type: Different approaches.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!pip install nltk

In [10]:
import pandas as pd
import json
import gensim
import nltk
from nltk.corpus import stopwords
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
# Lib Metrics
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [13]:
#install stopwords
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Unzipping corpora/stopwords.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [70]:
prefix = './drive/MyDrive/datasets/english/'
fake_ruta = prefix + 'fake_clean.json'
true_ruta = prefix + 'real_clean.json'
fake2_ruta = prefix + 'gpt2.json'

fake_news = pd.read_json(fake_ruta, lines=True)
fake_news['tag'] = 0
true_news = pd.read_json(true_ruta, lines=True)
true_news['tag'] = 1
fake_news_2 = pd.read_json(fake2_ruta, lines=True)
fake_news_2['tag'] = 0
frames = [fake_news, true_news, fake_news_2]
df = pd.concat(frames)
df.head(10)

,text,tag
0,A post claims compulsory vacination violates t...,0
1,A photo claims that this person is a doctor wh...,0
2,Post about a video claims that it is a protest...,0
3,All deaths by respiratory failure and pneumoni...,0
4,The dean of the College of Biologists of Euska...,0
5,Households with COVID-19 patients in Porto Ale...,0
6,A chain lists recommendations to prevent and t...,0
7,"60,000 Argentinian companies have closed due t...",0
8,Social media posts criticize a photo from smil...,0
9,CDC has released an update on how the novel co...,0


## Processing

In [71]:
import re
import string
stop = stopwords.words('english')

def review_cleaning(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
df['text']=df['text'].apply(lambda x:review_cleaning(x))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df

,text,tag
0,post claims compulsory vacination violates pri...,0
1,photo claims person doctor died attending many...,0
2,post video claims protest confination town ara...,0
3,deaths respiratory failure pneumonia registere...,0
4,dean college biologists euskadi states lot pcr...,0
...,...,...
2185,bill gates patented vaccine according one repo...,0
2186,virus weakens means artificial governor veneto...,0
2187,people violate social distancing protocol temp...,0
2188,image doctor smoking treating covid patients g...,0


In [72]:
from tensorflow.keras.preprocessing.text import one_hot
#One hot encoding 
voc_size = 14535
onehot_repr=[one_hot(words,voc_size)for words in df['text']]

## Splitting Datasets

In [73]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Padding the sentences
embedded_docs=pad_sequences(onehot_repr,padding='pre')
print(embedded_docs.shape)

(19179, 64)


In [74]:
# Converting the X and y as array
X_final=np.array(embedded_docs)
y_final=np.array(df['tag'])

#Check shape of X and y final
X_final.shape,y_final.shape

((19179, 64), (19179,))

In [75]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.33, random_state=42)

## Model

In [76]:
#Creating the lstm model
sent_length = 5000
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100)) #Adding 100 lstm neurons in the layer
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

#Compiling the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 5000, 40)          581400    
_________________________________________________________________
dropout_12 (Dropout)         (None, 5000, 40)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_13 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 637,901
Trainable params: 637,901
Non-trainable params: 0
_________________________________________________________________
None


In [77]:
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=10,batch_size=64)

Epoch 1/10
135/135 [==============================] - 16s 103ms/step - loss: 0.4519 - accuracy: 0.7757 - val_loss: 0.2048 - val_accuracy: 0.9293
Epoch 2/10
135/135 [==============================] - 13s 99ms/step - loss: 0.1371 - accuracy: 0.9488 - val_loss: 0.1765 - val_accuracy: 0.9335
Epoch 3/10
135/135 [==============================] - 13s 98ms/step - loss: 0.0991 - accuracy: 0.9662 - val_loss: 0.1686 - val_accuracy: 0.9385
Epoch 4/10
135/135 [==============================] - 13s 98ms/step - loss: 0.0463 - accuracy: 0.9859 - val_loss: 0.1851 - val_accuracy: 0.9382
Epoch 5/10
135/135 [==============================] - 13s 98ms/step - loss: 0.0291 - accuracy: 0.9912 - val_loss: 0.1981 - val_accuracy: 0.9403
Epoch 6/10
135/135 [==============================] - 13s 98ms/step - loss: 0.0167 - accuracy: 0.9954 - val_loss: 0.2402 - val_accuracy: 0.9387
Epoch 7/10
135/135 [==============================] - 13s 98ms/step - loss: 0.0110 - accuracy: 0.9970 - val_loss: 0.2471 - val_accuracy

## Evaluation of the model

In [78]:
# Predicting from test data
y_pred=model.predict_classes(X_test)


#Creating confusion matrix
#confusion_matrix(y_test,y_pred)
cm = metrics.confusion_matrix(y_test, y_pred)
# plot_confusion_matrix(cm,classes=['Fake','True'])
print(cm)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[3769  161]
 [ 252 2148]]


In [79]:
accuracy_score(y_test,y_pred)

0.9347551342812006

In [82]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      3930
           1       0.93      0.90      0.91      2400

    accuracy                           0.93      6330
   macro avg       0.93      0.93      0.93      6330
weighted avg       0.93      0.93      0.93      6330



In [81]:
voc_size = 14535
onehot_repr=[one_hot(words,voc_size)for words in ['Covid va a matar a mucha gente']]
embedded_docs=pad_sequences(onehot_repr,padding='pre')
model.predict(embedded_docs, batch_size=64)

array([[0.99964523]], dtype=float32)